# Tutorial 01: Running OpenDSS Simulations

This tutorial walks through the process of running non-RL grid simulations in PyCIGAR.

Keypoints: kernel, PyCIGAR API, run a simulation step.

## 0. OpenDSS

OpenDSS is an electric power Distribution System Simulator (DSS) for supporting distributed resource integration and grid modernization efforts.

PyCIGAR uses OpenDSS as a grid simulator. To interact with OpenDSS, we use the OpenDSSDirect, a OpenDSS Python API to interact with OpenDSS. A documentation can be found [here](http://dss-extensions.org/OpenDSSDirect.py/). Below is a sample code interacting with OpenDSS through OpenDSSDirect.

In [1]:
import opendssdirect as dss
# load the pre-defined grid to opendss, we use the grid sample in /pycigar/tutorials/data/ieee3.dss
# this grid is really simple and only contains 3 loads for the purpose of demonstration
dss.run_command("Redirect ./data/ieee3.dss")

# get the list of load names to see if the grid is loaded sucessfully
print(dss.Loads.AllNames())

['s701a', 's702a', 's703a']


In the code above, we import `opendssdirect` library, load the pre-defined grid to `opendss` and get a list of all the loads in the grid. More modules [here](http://dss-extensions.org/OpenDSSDirect.py/opendssdirect.html).

> **Note**: we import `opendssdirect` and call functions from it instead of creating an opendss object and interact with that opendss object **(!)**. As declaring by `opendssdirect`, it is an interface to opendss. This implies that if anywhere in the code we import the opendssdirect again (assume on the same computer process), we would get the interface and interact with the same opendss object. Because of this implementation, to have 2 distinct opendss instances, we need to create 2 distinct processes and call `opendssdirect` on these processes. 

> Each PyCIGAR environment requires 1 grid solver (opendss). Because we could not create multiple opendss objects on the same process, therefore each PyCIGAR environment has to create on a separate process. Most of the time, if you do not intend to train RL agent(s), you should not have to worry about creating multiple environments. 

## 1. Components of PyCIGAR
### Kernel
In all simulations of PyCIGAR, the high-level process is:

1. Initialize components (e.g. controllers, devices, nodes,...)
2. Run components
3. Update components' values to the simulator (e.g. new load value at a node)
4. Run the simulator (e.g. opendss)
5. Update components (with new values from the simulator e.g. voltages,...)
6. Back to step 2 until finish     
     
We need a central object to cordinate the process, it is responsible for all major activities in PyCIGAR. The central object is called `Kernel`. The `Kernel` has 4 `sub-kernels`:
- `Simulation kernel`: manage the interaction with simulator. The Simulation kernel returns the PyCIGAR API to interact with the simulator. 
- `Device kernel`: manage all the devices in the grid. The Device kernel has a dictionary of devices in the grid: the device, the node that the device is connected to, and controllers that control the device. Each device is given a `device_id`.  
- `Node kernel`: manage all the nodes in the grid. The Node kernel has a dictionary of nodes in the grid: the voltage profile at the node, the load profile at the node and the power, reactive power injection at a timestep. Each node is given a `node_id` (`node_id` is the same as load name in OpenDSS).
- `Scenario kernel`: manage dynamical change at the beginning or during the simulation. For example, Scenario kernel change the load and solar profiles at the beginning if the simulation, update the controllers to enable hack at an inverter.  

The code below shows how to create a `Kernel`.

In [2]:
from pycigar.core.kernel.kernel import Kernel

k = Kernel(simulator="opendss", sim_params=None)

# get the 4 sub-kernels
k_simulation = k.simulation
k_scenario = k.scenario
k_node = k.node
k_device = k.device
#####################################
# get the kernel from the sub-kernels
#these functions below should return the same kernel object
k1 = k_simulation.master_kernel
k2 = k_scenario.master_kernel
k3 = k_node.master_kernel
k4 = k_device.master_kernel

# assert that we get the same kernel object
assert(k1 == k2 == k3 == k4)

In the code above, we [instantiate a kernel object `k`](https://github.com/lbnl-cybersecurity/ceds-cigar/blob/toan_dev/pycigar/core/kernel/kernel.py#L44) with simulator "opendss". The kernel invoke 4 sub-kernels we mentioned above. From the sub-kernels, we can get the kernel by attibute [`master_kernel`](https://github.com/lbnl-cybersecurity/ceds-cigar/blob/toan_dev/pycigar/core/kernel/simulation/base.py#L8).  

### PyCIGAR API
Even though OpenDSSDirect is the API to interact with OpenDSS, PyCIGAR provides PyCIGAR API to interact with the OpenDSS through OpenDSSDirect. The purpose is to standarize the API call across PyCIGAR so we could intergrate new simulator API quickly by wrapping the new simulator API with PyCIGAR API. To get the PyCIGAR API for OpenDSS:

In [3]:
kernel_api = k.simulation.start_simulation()

The `kernel_api` is an object define [here](https://github.com/lbnl-cybersecurity/ceds-cigar/blob/toan_dev/pycigar/utils/opendss/pseudo_api.py). We need to pass the `kernel_api` to 4 sub-kernels, so they could get and set information from OpenDSS.

In [4]:
k_simulation.pass_api(kernel_api)
k_scenario.pass_api(kernel_api)
k_node.pass_api(kernel_api)
k_device.pass_api(kernel_api)

# or just simply call 
k.pass_api(kernel_api)

We has instantiated kernel, get PyCIAR API to interact with OpenDSS. In the next section, we will learn how to feed the feeder to the simulator, add devices and change the load solar profiles. 

### PyCIGAR configurations file

PyCIGAR configurations file define essential information to run a simulation (e.g. the directory of opendss file and load-solar profiles, list of installed devices at nodes,...). The configuration file for this tutorial [here](./data/pycigar_config.yaml).
- `simulation_config`: contains information specific to opendss.
- `scenario_config`: contrains the information about the installed devices and controllers. 
  - `network_data_directory`: directory to the load-solar profiles.
  - `start_end_time`: list of start time and end time. This is the index number in load-solar profiles file `.csv`.  For example, if we want to run the experiment with load-solar profiles from line 100 to line 300, put here `[100, 300]`.
  - `custom_configs`: contains additional parameters applied across the nodes.
  - `nodes`: each node define by name (has to match with name in opendss in lowercase) and `devices` define the list of devices installed at that node. For the example below, node `s701a` has an PV inverter installed, the name of this device is `pv_1`. Whenever we have an device at a node, we also have an adversarial device of that device at the same node, the device identification is begin with `adversary`. For example in this case: `adversary_pv_1`. The node `s702a` and `s703a` do not have any devices thus you can delete them in the PyCIGAR configuration file. We let them here for demonstration purpose.

```yaml
simulation_config:
  network_model_directory: './data/ieee3.dss'
  custom_configs:  {solution_mode: 1,
                    solution_number: 1,
                    solution_step_size: 1,
                    solution_control_mode: -1,
                    solution_max_control_iterations: 1000000,
                    solution_max_iterations: 30000,
                    power_factor: 0.9}

scenario_config:
  network_data_directory: './data/load_solar_data.csv'
  start_end_time: [100, 500]
  custom_configs: {load_scaling_factor: 1.5,
                   solar_scaling_factor: 3,
                   slack_bus_voltage: 1.04,
                   load_generation_noise: False,
                   power_factor: 0.9}

  nodes:
      - name: s701a
        devices:
            - name: pv_1
              type: pv_device
              controller: fixed_controller
              custom_configs: {default_control_setting: [0.98, 1.01, 1.01, 1.04, 1.06],
                               low_pass_filter_measure: 1.2,
                               low_pass_filter_output: 0.115}
              adversary_controller: fixed_controller
              adversary_custom_configs: {default_control_setting: [1.014, 1.015, 1.015, 1.016, 1.017]}
              hack: [300, 0.4]
      - name: s702a

      - name: s703a
```

Let's read `.yaml` file to `sim_params` and create the `Kernel` again, this time with `sim_params`.

In [1]:
from pycigar.core.kernel.kernel import Kernel
import yaml

stream = open("./data/pycigar_config.yaml", "r")
sim_params = yaml.safe_load(stream)

k = Kernel(simulator="opendss", sim_params=sim_params)
kernel_api = k.simulation.start_simulation()
k.pass_api(kernel_api)

In [2]:
k.update(reset=True)